In [ ]:
#---------------------------------------------------BASIC STATISTICS-2------------------------------------------------------------------

In [ ]:
# Context:
# A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.
# Tasks:
# 1.	Load the patient dataset and show summary with info().
# 2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
# 3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
# 4.	Use groupby to find total bill amount per department.
# 5.	Remove duplicate patient records based on PatientID.
# 6.	Fill missing BillAmount values with the mean bill amount.
# 7.	Merge the billing dataset with patient dataset on PatientID.
# 8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
# 9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).


In [3]:
import pandas as pd
import numpy as np

In [4]:
patient_df = pd.read_csv("C:\\Users\\sdsum\\OneDrive\\Desktop\\EXCELR\\ASSIGNMENTS\\CSV FILES\\4.2 Patient_Data.csv")

In [5]:
#Summary of the data
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [6]:
patient_df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [7]:
#Select only columns relevant for billing

billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
billing_df = patient_df[billing_cols]

billing_df.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [8]:
# Drop administrative columns like ['ReceptionistID', 'CheckInTime']
cleaned_df = patient_df.drop(columns=['ReceptionistID', 'CheckInTime'], errors='ignore')

cleaned_df.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [9]:
# 4.	Use groupby to find total bill amount per department
dept_bill = cleaned_df.groupby('Department')['BillAmount'].sum().reset_index()

print("Total Bill Amount per Department:")
print(dept_bill)

Total Bill Amount per Department:
    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


In [14]:
# 5.	Remove duplicate patient records based on PatientID.
unique_patients = cleaned_df.drop_duplicates(subset='PatientID')

print("After removing duplicates:")
print(unique_patients.shape)


After removing duplicates:
(5, 5)


In [16]:
# 6.	Fill missing BillAmount values with the mean bill amount
mean_bill = unique_patients['BillAmount'].mean()


unique_patients = unique_patients.copy() 
unique_patients['BillAmount'] = unique_patients['BillAmount'].fillna(mean_bill)

print("Missing BillAmount values filled with mean:", mean_bill)
print(unique_patients.isnull().sum())


Missing BillAmount values filled with mean: 6233.333333333333
PatientID     0
Name          0
Department    0
Doctor        0
BillAmount    0
dtype: int64


In [18]:
# 7: Merge the billing dataset with patient dataset on PatientID
billing_details = pd.read_csv("C:\\Users\\sdsum\\OneDrive\\Desktop\\EXCELR\\ASSIGNMENTS\\CSV FILES\\4.1 Billing_Data.csv") 
merged_df = pd.merge(unique_patients, billing_details, on='PatientID', how='left')

print("Merged Dataset:")
merged_df.head()

Merged Dataset:


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000


In [19]:
# 8: Concatenate an additional DataFrame with new patients (row-wise)

new_patients = pd.DataFrame({
    'PatientID': [201, 202],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. A', 'Dr. B'],
    'BillAmount': [15000, 20000]
})

updated_df = pd.concat([merged_df, new_patients], ignore_index=True)

print("After adding new patient rows:")
updated_df.tail()

After adding new patient rows:


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0
5,201,NaN,Cardiology,Dr. A,15000.000000,NaN,NaN
6,202,NaN,Neurology,Dr. B,20000.000000,NaN,NaN


In [22]:
# 9.Concatenate new billing category columns (column-wise)

import numpy as np
insurance_col = ['Yes', 'No'] * (len(updated_df) // 2) + ['Yes'] * (len(updated_df) % 2)

extra_cols = pd.DataFrame({
    'InsuranceCovered': insurance_col,
    'FinalAmount': updated_df['BillAmount'] * 0.9
})

final_df = pd.concat([updated_df.reset_index(drop=True), extra_cols], axis=1)

print("Final Cleaned Dataset:")
print(final_df.head())


Final Cleaned Dataset:
   PatientID     Name   Department     Doctor   BillAmount InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000           2000.0   
1        102      Bob    Neurology   Dr. John  6233.333333           1500.0   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000           2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000           3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333           1000.0   

   FinalAmount InsuranceCovered  FinalAmount  
0       3000.0              Yes       4500.0  
1       3500.0               No       5610.0  
2       5000.0              Yes       6750.0  
3       3200.0               No       5580.0  
4       4000.0              Yes       5610.0  
